In [1]:
import re
from collections import deque

In [2]:
# global variables
obstacles = []
nets = {}
used_paths = set() 
ROWS = COLS = VIA_COST = NON_PREF_COST = 0

<h2> Parsing </h2>

In [4]:
def parse_input_file(file_path):
    
    with open(file_path) as f:
        global ROWS, COLS, VIA_COST, NON_PREF_COST
        values = f.readline().strip()
        ROWS, COLS, VIA_COST, NON_PREF_COST = map(int, values.split(','))

        for line in f:
            line = line.strip()
            if line.startswith("OBS"):
                obstacles.append(tuple(map(int, re.findall(r"\d+", line))))
            elif line.startswith("net"):
                net_name, coords = line.split(' ', 1)
                nets[net_name] = [tuple(map(int, re.findall(r"\d+", coord))) for coord in coords.split(') (')]

In [5]:
def print_input_file_info():
    print(f"Rows: {ROWS}\nCols: {COLS}\nVia Cost: {VIA_COST}\nNon-preferred Direction Cost: {NON_PREF_COST}\n")
    
    print("Obstacles:")
    for obs in obstacles:
        print(f"  {obs}")
    
    print("\nNets:")
    for net, coords in nets.items():
        print(f"  {net}: {coords}")

In [6]:
parse_input_file('input.txt')
print_input_file_info()

Rows: 3
Cols: 3
Via Cost: 5
Non-preferred Direction Cost: 2

Obstacles:
  (1, 1)

Nets:
  net1: [(0, 0, 0), (0, 2, 2)]


<h2> Routing </h2>

In [8]:

def bfs(start, end):
   
    queue = deque([(start[1], start[2], None)])  # (x, y, dir where it came from)
    
    visited = set()
    
    grid = [[float('inf')] * COLS for _ in range(ROWS)]  
    grid[start[1]][start[2]] = 0 
    
    DIRECTIONS = [(0, 1), (0, -1), (1, 0), (-1, 0)] 
    
    while queue:
        x, y, prev_direction = queue.popleft()

        if (x, y) in visited or (x, y) in obstacles:
            continue

        visited.add((x, y))

        if (x, y) == (end[1], end[2]):
            print(grid)
            return grid[end[1]][end[2]]  # return cost as we reached end

        for dx, dy in DIRECTIONS:
            nx, ny = x + dx, y + dy
            if 0 <= nx < ROWS and 0 <= ny < COLS: 
                if (nx, ny) in obstacles or (nx, ny) in visited:
                    continue

                if prev_direction is not None:
                    if (dx == 0 and prev_direction == 'H') or (dy == 0 and prev_direction == 'V'):
                        new_cost = grid[x][y] + VIA_COST + 1  # we change direction
                    else:
                        new_cost = grid[x][y] + 1  # same directoin
                else:
                    new_cost = grid[x][y] + 1  
                
                if new_cost < grid[nx][ny]:
                    grid[nx][ny] = new_cost
                    queue.append((nx, ny, 'H' if dy == 0 else 'V'))  
    return -1 


In [9]:
def route_nets():
    for net_name, coords in nets.items():
        
        start = coords[0]  # (layer, x, y)
        end = coords[1]  # (layer, x, y)
        
        print(f"Routing {net_name} from {start} to {end}")
        
        cost = bfs(start, end)
        
        if cost != -1:
            print(f"Route found for {net_name} with cost: {cost}")
        else:
            print(f"No route found for {net_name}.")

In [10]:
route_nets()

Routing net1 from (0, 0, 0) to (0, 2, 2)
[[0, 1, 2], [1, inf, 8], [2, 8, 9]]
Route found for net1 with cost: 9
